- 사용자의 질문을 처리하기 전에 검색 대상 문서를 사전에 인덱싱해야 합니다.
- 구글의 코드 스타일 가이드 문서를 크롤링하고 이를 기반으로 검색 가능한 벡터 DB를 구축합니다.


In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

#크롤링할 블로그의 url 정의
urls = [
    "https://google.github.io/styleguide/pyguide.html",
    "https://google.github.io/styleguide/javaguide.html",
    "https://google.github.io/styleguide/jsguide.html",
]

# WebBaseLoader를 사용하여 주어진 URL 목록에서 문서 크롤링
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# 지정한 크기만큼 텍스트를 분할하는 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

# 문서 분할
doc_splits = text_splitter.split_documents(docs_list)

# Chroma 벡터 저장소에 문서의 분할된 조각 저장
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OllamaEmbeddings(model="bge-m3")
)
#벡터 저장소에서 검색을 수행할 수 있는 검색기 생성
retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


- 문서 평가하기


In [2]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

#문서와 질문의 연관성을 평가하기 위한 데이터 모델 정의
class GradeDocuments(BaseModel):
    binary_score: str = Field(
        description="문서와 질문의 연관성 여부. (예 or 아니오)"
    )

# 연관성 평가를 위한 LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# LLM이 사용자의 질문에 대해 문서의 연관성을 평가할 수 있도록 지시하는 프롬프트 정의
system = """당신은 사용자의 질문에 대해 검색된 문서의 관련성을 평가하는 전문가입니다.
문서에 질문과 관련된 키워드나 의미가 담겨 있으면, 해당 문서를 '관련 있음'으로 평가하세요.
문서가 질문과 관련이 있는지 여부를 '예' 또는 '아니오'로 표시해 주세요."""

# 시스템 메시지와 사용자의 질문 및 문서 내용을 포함한 템플릿 작성
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "검색된 문서:\n\n {document}\n\n 사용자 질문: {question}"),
    ]
)

# 프롬프트와 구조화된 LLM 평가기를 결합하여 retrieval_grader 객체 생성
retrieval_grader = grade_prompt | structured_llm_grader

- 질문과 연관된 문서인지 연관성 평가를 해보겠습니다.


In [4]:
question = "파이썬 코드 작성 가이드"
# 연관 문서 검색
docs = retriever.invoke(question)
doc_txt = docs[0].page_content

# 검색된 문서와 질문을 사용하여 연관성 평가 수행
print(retrieval_grader.invoke({"document": doc_txt, "question": question}))

binary_score='예'


- 답변 생성하기


In [5]:
from langchain_core.output_parsers import StrOutputParser

# LLM이 제공된 문맥을 바탕으로 답변할 수 있도록 지시하는 프롬프트 정의
system = """당신은 질문에 답변하는 업무를 돕는 도우미입니다.
제공된 문맥을 바탕으로 질문에 답변하세요. 만약 답을 모르면 모른다고 말하세요.
세 문장을 넘지 않도록 답변을 간결하게 작성해주세요."""

# 시스템 메시지와 사용자의 질문 및 문서 내용을 포함한 템플릿 작성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "질문: {question}\n\n문맥: {context}\n\n답변:"),
    ]
)

# 검색된 문서들을 한 문자열로 병합
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# 프롬프트, LLM, 문자열 출력을 결합하여 RAG 체인 생성
rag_chain = prompt | llm | StrOutputParser()

In [6]:
# 정의된 RAG 체인을 사용하여 질문과 문맥을 기반으로 답변 생성
generation = rag_chain.invoke({"context": format_docs(docs), "question": question})

# 생성된 답변 출력
print(generation)

Google Python Style Guide는 파이썬 코드 작성 시 따를 수 있는 규칙과 지침을 제공합니다. 새로운 파일은 Google 스타일을 사용해야 하며, 기존 코드와의 일관성을 유지하는 것이 중요합니다. 추가적인 스타일 규칙은 팀이나 프로젝트에 따라 채택할 수 있지만, 과도한 규칙은 피해야 합니다.
